In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_path = "/kaggle/input/tabular-playground-series-aug-2021/train.csv"
test_path = "/kaggle/input/tabular-playground-series-aug-2021/test.csv"
sample_path = "/kaggle/input/tabular-playground-series-aug-2021/sample_submission.csv"

In [ ]:
import gc
import os
import shutil
import datatable as dt
from pathlib import Path

train = dt.fread(train_path)
test = dt.fread(test_path)

target = train['loss'].to_numpy().ravel()

del train[:, ['id', 'loss']]
test_ids = test[:, 'id']
test = test[:, train.names]

In [ ]:
!python3 -m pip install -q lightautoml

In [ ]:
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task

In [ ]:
train['target'] = dt.Frame(target)

model = TabularAutoML(
    task=Task('reg'),
    timeout=500,
    verbose=2
)

model.fit_predict(train_data=train.to_pandas(), roles={'target': 'target'})

del train['target']

In [ ]:
preds = model.predict(test.to_pandas()).data.ravel()

In [ ]:
submission = dt.Frame(
    id=test_ids,
    loss=preds
)

submission.head()

In [ ]:
submission.to_csv("submission.csv")